In [19]:
import pandas as pd
import spacy
import csv
import string as str
import json

## Aperçu des données

28 834 entrées et 46 variables par entrée

In [3]:
df = pd.read_csv("DEMOCRATIE_ET_CITOYENNETE.csv", low_memory = False)
df.shape

df.head()



ParserError: ignored

## Extraction des données

https://docs.python.org/fr/3/library/csv.htmlhttps://docs.python.org/fr/3/library/csv.html

In [ ]:
def ponctuation(texte):
    for p in str.punctuation:
        texte = texte.replace(p, " ")
    return texte

In [5]:
#Extraction des données dans une liste de listes
#len(ex) = 28385 soit le nombre d’observation

observations = []
with open("DEMOCRATIE_ET_CITOYENNETE.csv", newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row != []:
            observations.append(row)

## POS-Tagging using SpaCy

In [8]:
#NE PAS RUN cette cellule sauf sur Google Colab

try:
  nlp = spacy.load("fr_core_news_sm", disable=['ner'])
except:
  from spacy.cli import download
  download('fr_core_news_sm')
  nlp = spacy.load("fr_core_news_sm", disable=['ner'])

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [9]:
nlp = spacy.load("fr_core_news_sm")
#doc = nlp(d) # /!\ Doit être de type String

doc = []
nouns = []
adj = []
pronouns = []
for obs in observations:
    for sent in obs:
        d = nlp(sent)
        for token in d:
            if token.pos_ == "NOUN" : nouns.append((token.text, token.pos_))
            elif token.pos_ == "ADJ" : adj.append((token.text, token.pos_))
            elif token.pos_ == "PRON" : pronouns.append((token.text, token.pos_))
            #doc.append((token.text, token.pos_))

In [17]:
print(len(nouns))

399875
399875


In [ ]:
print(adj)

[('publishedAt', 'ADJ'), ('faites', 'ADJ'), ('politiques', 'ADJ'), ('important', 'ADJ'), ('syndicales', 'ADJ'), ('professionnelles', 'ADJ'), ('sénateurs', 'ADJ'), ('différentes', 'ADJ'), ('politiques', 'ADJ'), ('souhaitable', 'ADJ'), ('sénateurs', 'ADJ'), ('blanc', 'ADJ'), ('grandes', 'ADJ'), ('publique', 'ADJ'), ('électoral', 'ADJ'), ('applicable', 'ADJ'), ('public', 'ADJ'), ('économique', 'ADJ'), ('social', 'ADJ'), ('environnemental', 'ADJ'), ('civile', 'ADJ'), ('tous', 'ADJ'), ('réciproque', 'ADJ'), ('intangibles', 'ADJ'), ('Quels', 'ADJ'), ('civiques', 'ADJ'), ('quotidienne', 'ADJ'), ('collective', 'ADJ'), ('civiques', 'ADJ'), ('quels', 'ADJ'), ('concrets', 'ADJ'), ('publics', 'ADJ'), ('Quelles', 'ADJ'), ('pénibles', 'ADJ'), ('quotidienne', 'ADJ'), ('publics', 'ADJ'), ('Quelles', 'ADJ'), ('répandues', 'ADJ'), ('victime', 'ADJ'), ('solidaire', 'ADJ'), ('tolérante', 'ADJ'), ('différentes', 'ADJ'), ('migratoire', 'ADJ'), ('Quelles', 'ADJ'), ('migratoire', 'ADJ'), ('annuels', 'ADJ'), (

In [ ]:
print(pronouns)

[('trashedStatus', 'PRON'), ('qui', 'PRON'), ('-vous', 'PRON'), ('vous', 'PRON'), ('-il', 'PRON'), ('Que', 'PRON'), ('-il', 'PRON'), ('qui', 'PRON'), ('les', 'PRON'), ('Que', 'PRON'), ('-il', 'PRON'), ('-vous', 'PRON'), ('il', 'PRON'), ('lesquels', 'PRON'), ('Que', 'PRON'), ('-vous', 'PRON'), ('les', 'PRON'), ('y', 'PRON'), ('-il', 'PRON'), ('Que', 'PRON'), ('-il', 'PRON'), ('-il', 'PRON'), ('qui', 'PRON'), ('Que', 'PRON'), ('-il', 'PRON'), ('dont', 'PRON'), ('elles', 'PRON'), ('-il', 'PRON'), ('les', 'PRON'), ('Que', 'PRON'), ('Que', 'PRON'), ('-il', 'PRON'), ('il', 'PRON'), ('Que', 'PRON'), ('-il', 'PRON'), ('chacun', 'PRON'), ('-il', 'PRON'), ('y', 'PRON'), ('Que', 'PRON'), ('-il', 'PRON'), ('-il', 'PRON'), ('Que', 'PRON'), ('chacun', 'PRON'), ('dont', 'PRON'), ('vous', 'PRON'), ('Que', 'PRON'), ('-il', 'PRON'), ('-vous', 'PRON'), ('il', 'PRON'), ('lesquelles', 'PRON'), ('Que', 'PRON'), ('-vous', 'PRON'), ('vous', 'PRON'), ('-vous', 'PRON'), ('nous', 'PRON'), ('nous', 'PRON'), ('Que

## Sérialisation des données

Données sérialisées au format json car facilement lisible hors programmation  

https://www.codeflow.site/fr/article/python-json  

La cellule ci-dssous ne doit être runé qu'une seule fois

In [22]:
with open("nouns.json", "w+") as fichier:
  json.dump(nouns, fichier)

with open("adjectives.json", "w+") as fichier:
  json.dump(adj, fichier)
 
with open("pronouns.json", "w+") as fichier:
  json.dump(pronouns, fichier)
  #fichier.write(pronouns)